# Flash Evaluation on the DARPA OpTC Dataset

This notebook is designed for evaluating Flash on the DARPA OpTC dataset. The OpTC dataset is a node-level dataset, crucial for our analysis. Flash is configured to operate in a node-level setting to effectively assess this dataset. The OpTC dataset is enriched with node attributes, making it suitable for running Flash in a decoupled manner. This includes using offline GNN embeddings and a downstream classifier. Our approach tests Flash on this dataset, where Flash generates word2vec embeddings as feature vectors for GNN embeddings. These embeddings are stored in a datastore and used in conjunction with a downstream model for improved detection results.

## Accessing the Dataset:
- The OpTC dataset can be accessed via this link: [OpTC Dataset](https://drive.google.com/drive/u/0/folders/1n3kkS3KR31KUegn42yk3-e6JkZvf0Caa).
- Dataset files for evaluation will be downloaded automatically by the script.
- While we provide pre-trained weights, you also have the option to download benign data files for training the models from the ground up.

## Data Parsing and Execution:
- The script is adept at autonomously parsing the downloaded data files.
- For evaluation results, execute all cells in this notebook.

## Model Training and Execution Options:
- By default, the notebook utilizes pre-trained model weights.
- It also offer settings to independently train Graph Neural Networks (GNNs), word2vec, and Xgboost models.
- These independently trained models can then be deployed for an evaluation of the system.

Following these guidelines will ensure a thorough and effective analysis of the OpTC dataset using Flash.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data
import os
import torch.nn.functional as F
import pickle
import json
import warnings
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
warnings.filterwarnings('ignore')
from torch_geometric.loader import NeighborLoader
import csv

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cuda:5")
csv_path = "../../flash-add-edge/optc-201.csv"

import subprocess
gpu_mem = {int(x.split(',')[0]): int(x.split(',')[1]) for x in subprocess.check_output(
    ["nvidia-smi", "--query-gpu=index,memory.free", "--format=csv,noheader,nounits"], 
    encoding='utf-8').strip().split('\n')}
best_gpu = max(gpu_mem.items(), key=lambda x: x[1])[0]
device = torch.device(f'cuda:{best_gpu}' if torch.cuda.is_available() else 'cpu')
print(device)

%matplotlib inline

cuda:2


In [2]:
# gnn_weights = "trained_weights/optc/gnn_temp.pth"
gnn_weights = "gnn_temp.pth"
# xgboost_weights = "trained_weights/optc/xgb.pkl"
xgboost_weights = "xgb.pkl"
word2vec_weights = 'w2v_optc.model'
create_store = True
gnnTrain = True
xgbTrain = True
w2vTrain = True

In [3]:
from pprint import pprint
import gzip
from sklearn.manifold import TSNE
import json
import copy
import os
import xgboost as xgb

import gensim
from gensim.models import Word2Vec
from multiprocessing import Pool
from itertools import compress
from tqdm import tqdm
import time

import multiprocessing

In [4]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import Counter
from gensim.models import Word2Vec
from multiprocessing import Pool
from itertools import compress
from tqdm import tqdm
import time

In [5]:
import gzip
import io

def extract_logs(filepath, hostid):
    search_pattern = f'../SysClient{hostid}'
    output_filename = f'../SysClient{hostid}.systemia.com.txt'
    
    with gzip.open(filepath, 'rt', encoding='utf-8') as fin:
        with open(output_filename, 'ab') as f:
            out = io.BufferedWriter(f)
            for line in fin:
                if search_pattern in line:
                    out.write(line.encode('utf-8'))
            out.flush()

In [6]:
# import gdown
from tqdm import tqdm
    
def prepare_test_set():
    # urls = [
    #     "https://drive.google.com/file/d/1HFSyvmgH0jvdnnnTdKfWRjZYOrLWoIkv/view?usp=drive_link",
    #     "https://drive.google.com/file/d/1pJLxJsDV8sngiedbfVajMetczIgM3PQd/view?usp=drive_link",
    #     "https://drive.google.com/file/d/1fRQqc68r8-z5BL7H_eAKIDOeHp7okDuM/view?usp=drive_link",
    #     "https://drive.google.com/file/d/1VfyGr8wfSe8LBIHBWuYBlU8c2CyEgO5C/view?usp=drive_link",
    #     "https://drive.google.com/file/d/10N9ZPolq_L8HivBqzf_jFKbwjSxddsZp/view?usp=drive_link",
    #     "https://drive.google.com/file/d/1xIr8gw-4zc8ESjUpYtrFsbOwhPGUSd15/view?usp=drive_link",
    #     "https://drive.google.com/file/d/1PvlCp2oQaxEBEFGSQWfcFVj19zLOe7yH/view?usp=drive_link"
    # ]

    # for url in urls:
    #     gdown.download(url, quiet=False, use_cookies=False, fuzzy=True)

    log_files = [
        ("AIA-201-225.ecar-2019-12-08T11-05-10.046.json.gz", "0201"),
        ("AIA-201-225.ecar-last.json.gz", "0201"),
        ("AIA-501-525.ecar-2019-11-17T04-01-58.625.json.gz", "0501"),
        ("AIA-501-525.ecar-last.json.gz", "0501"),
        ("AIA-51-75.ecar-last.json.gz", "0051")
    ]
    
    os.system("rm SysClient0201.com.txt")
    os.system("rm SysClient0501.com.txt")
    os.system("rm SysClient0051.com.txt")

    for file, code in tqdm(log_files, desc="Extracting logs", unit="file"):
        extract_logs(file, code)

# prepare_test_set()

In [7]:
def is_valid_entry(entry):
    valid_objects = {'PROCESS', 'FILE', 'FLOW', 'MODULE'}
    invalid_actions = {'START', 'TERMINATE'}

    object_valid = entry['object'] in valid_objects
    action_valid = entry['action'] not in invalid_actions
    actor_object_different = entry['actorID'] != entry['objectID']

    return object_valid and action_valid and actor_object_different

def Traversal_Rules(data):
    filtered_data = {}

    for entry in data:
        if is_valid_entry(entry):
            key = (
                entry['action'], 
                entry['actorID'], 
                entry['objectID'], 
                entry['object'], 
                entry['pid'], 
                entry['ppid']
            )
            filtered_data[key] = entry

    return list(filtered_data.values())

In [8]:
def Sentence_Construction(entry):
    action = entry["action"]
    properties = entry['properties']
    object_type = entry['object']

    format_strings = {
        'PROCESS': "{parent_image_path} {action} {image_path} {command_line}",
        'FILE': "{image_path} {action} {file_path}",
        'FLOW': "{image_path} {action} {src_ip} {src_port} {dest_ip} {dest_port} {direction}",
        'MODULE': "{image_path} {action} {module_path}"
    }

    default_format = "{image_path} {action} {module_path}"

    try:
        format_str = format_strings.get(object_type, default_format)
        phrase = format_str.format(action=action, **properties)
    except KeyError:
        phrase = ''

    return phrase.split(' ')

In [9]:
import pandas as pd
import json

def Extract_Semantic_Info(event):
    object_type = event['object']
    properties = event['properties']

    label_mapping = {
        "PROCESS": ('parent_image_path', 'image_path'),
        "FILE": ('image_path', 'file_path'),
        "MODULE": ('image_path', 'module_path'),
        "FLOW": ('image_path', 'dest_ip', 'dest_port')
    }

    label_keys = label_mapping.get(object_type, None)
    if label_keys:
        labels = [properties.get(key) for key in label_keys]
        if all(labels):
            event["actorname"], event["objectname"] = labels[0], ' '.join(labels[1:])
            return event
    return None

def transform(text):
    labeled_data = [event for event in (Extract_Semantic_Info(x) for x in text) if event]
    data = Traversal_Rules(labeled_data)

    phrases = [Sentence_Construction(x) for x in data if Sentence_Construction(x)]
    for datum, phrase in zip(data, phrases):
        datum['phrase'] = phrase

    df = pd.DataFrame(data)
    df['timestamp'] = pd.to_datetime(df['timestamp'].str[:-6], infer_datetime_format=True)
    df.sort_values(by='timestamp', inplace=True)

    return df

def load_data(file_path):
    with open(file_path, 'r') as file:
        content = [json.loads(line) for line in file]
    
    return Featurize(transform(content), csv_path)

In [10]:
import pandas as pd
import numpy as np
from collections import defaultdict


def Featurize(df, additional_edges_file=None, thres=1000000000):
    dummies = {'PROCESS': 0, 'FLOW': 1, 'FILE': 2, 'MODULE': 3}

    nodes = {}
    labels = {}
    lblmap = {}
    neimap = {}
    edges = []
    for index, row in df.iterrows():
        actor_id, object_id = row['actorID'], row["objectID"]
        object_type = row['object']

        nodes.setdefault(actor_id, []).extend(row['phrase'])
        nodes.setdefault(object_id, []).extend(row['phrase'])

        labels[actor_id] = dummies.get('PROCESS', -1)
        labels[object_id] = dummies.get(object_type, -1)

        lblmap[actor_id] = row['actorname']
        lblmap[object_id] = row['objectname']

        neimap.setdefault(actor_id, set()).add(row['objectname'])
        neimap.setdefault(object_id, set()).add(row['actorname'])

        edge_type = row['properties']['direction'] if object_type == 'FLOW' else row['action']
        edges.append((actor_id, object_id, edge_type))

    if additional_edges_file:
        try:
            degrees = defaultdict(int)
            for src_id, dst_id, _ in edges:
                degrees[src_id] += 1
                degrees[dst_id] += 1
                
            all_degrees = list(degrees.values())
            all_degrees.sort(reverse=True)
            print(all_degrees[:3000])

            additional_edges_df = pd.read_csv(additional_edges_file, header=None, names=['src', 'dst'])
            
            count1, count2, count3 = 0, 0, 0
            for _, row in additional_edges_df.iterrows():
                src_id = str(row['src']).strip()
                dst_id = str(row['dst']).strip()

                if src_id in nodes and dst_id in nodes:
                    if degrees[src_id] <= thres and degrees[dst_id] <= thres:     
                        edges.append((src_id, dst_id, 'ADDITIONAL_EDGE'))
                        count1 += 1
                    else:
                        count2 += 1
                else:
                    count3 += 1
                
            print(f'Sucessfully add edges: {count1}\tPrune edges: {count2}\tFail to add edges: {count3}')
            
        except Exception as e:
            print(f"读取额外边文件时出错: {e}")


    features, feat_labels, edge_index = [], [], [[], []]
    node_index = {}

    
    for node, phrases in nodes.items():
        if not (len(phrases) == 1 and phrases[0] == 'DELETE'):
            features.append(infer(phrases))
            feat_labels.append(labels[node])
            node_index[node] = len(features) - 1

    
    for src, dst, _ in edges:
        if src in node_index and dst in node_index:
            edge_index[0].append(node_index[src])
            edge_index[1].append(node_index[dst])

    mapp = list(node_index.keys())

    return features, np.array(feat_labels), edge_index, mapp, lblmap, neimap

In [11]:
from gensim.models.callbacks import CallbackAny2Vec

class EpochSaver(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        model.save('w2v_optc.model')
        self.epoch += 1

In [12]:
class EpochLogger(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [13]:
import json
from gensim.models import Word2Vec

def prepare_sentences(df):
    nodes = {}
    for index, row in df.iterrows():
        for key in ['actorID', 'objectID']:
            node_id = row[key]
            nodes.setdefault(node_id, []).extend(row['phrase'])
    return list(nodes.values())

def train_word2vec_model(train_file_path):
    with open(train_file_path, 'r') as file:
        content = [json.loads(line) for line in file]

    events = transform(content)
    phrases = prepare_sentences(events)

    logger = EpochLogger()
    saver = EpochSaver()
    # word2vec = Word2Vec(sentences=phrases, vector_size=20, window=5, min_count=1, workers=8, epochs=300, callbacks=[saver, logger])
    word2vec = Word2Vec(sentences=phrases, vector_size=10, window=5, min_count=1, workers=8, epochs=300, callbacks=[saver, logger])

    return word2vec

In [14]:
import math
import torch
import numpy as np
from gensim.models import Word2Vec

class PositionalEncoder:

    def __init__(self, d_model, max_len=100000):
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        self.pe = torch.zeros(max_len, d_model)
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)

    def embed(self, x):
        return x + self.pe[:x.size(0)]


def infer(document):
    word_embeddings = [w2vmodel.wv[word] for word in document if word in w2vmodel.wv]
    
    if not word_embeddings:
        return np.zeros(10)
    
    output_embedding = torch.tensor(word_embeddings, dtype=torch.float)
    if len(document) < 100000:
        output_embedding = encoder.embed(output_embedding)

    output_embedding = output_embedding.detach().cpu().numpy()
    
    mean_embedding = np.mean(output_embedding, axis=0)
    
    return mean_embedding

# encoder = PositionalEncoder(20)
encoder = PositionalEncoder(10)
if w2vTrain:
    file_path = '../SysClient0201.systemia.com.txt'
    # 重新训练w2v
    w2vmodel = train_word2vec_model(file_path)
else:
    w2vmodel = Word2Vec.load(word2vec_weights)


Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
Epoch #10 start
Epoch #10 end
Epoch #11 start
Epoch #11 end
Epoch #12 start
Epoch #12 end
Epoch #13 start
Epoch #13 end
Epoch #14 start
Epoch #14 end
Epoch #15 start
Epoch #15 end
Epoch #16 start
Epoch #16 end
Epoch #17 start
Epoch #17 end
Epoch #18 start
Epoch #18 end
Epoch #19 start
Epoch #19 end
Epoch #20 start
Epoch #20 end
Epoch #21 start
Epoch #21 end
Epoch #22 start
Epoch #22 end
Epoch #23 start
Epoch #23 end
Epoch #24 start
Epoch #24 end
Epoch #25 start
Epoch #25 end
Epoch #26 start
Epoch #26 end
Epoch #27 start
Epoch #27 end
Epoch #28 start
Epoch #28 end
Epoch #29 start
Epoch #29 end
Epoch #30 start
Epoch #30 end
Epoch #31 start
Epoch #31 end
Epoch #32 start
Epoch #32 end
Epoch #33 start
Epoch #33 end


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        # self.conv1 = SAGEConv(20, 32, normalize=True)
        self.conv1 = SAGEConv(10, 32, normalize=True)
        self.conv2 = SAGEConv(32, 20, normalize=True)
        self.linear = nn.Linear(in_features=20, out_features=4)

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor) -> torch.Tensor:
    
        x = self.encode(x, edge_index)
        x = self.linear(x)
        return F.softmax(x, dim=1)
    
    def encode(self, x: torch.Tensor, edge_index: torch.Tensor) -> torch.Tensor:
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [16]:
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

model = GCN().to(device)
if not gnnTrain:
    model.load_state_dict(torch.load(gnn_weights))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [17]:
from sklearn.utils import class_weight


if gnnTrain or create_store:
    file_path = '../SysClient0201.systemia.com.txt'
    nodes,labels,edges,mapp,lbl,nemap = load_data(file_path)

    ngram_class_weights = [
        0.7667,  
        1.2574,  
        1.5000,  
        0.5000 
    ]
    class_weights = torch.tensor(ngram_class_weights,dtype=torch.float).to(device)
    
    criterion = CrossEntropyLoss(weight=class_weights,reduction='mean')

    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))

[72582, 8448, 6447, 4648, 3292, 2731, 2334, 2326, 2109, 2079, 1385, 1384, 1384, 1384, 1383, 1124, 1124, 1120, 1120, 1120, 1119, 1045, 1021, 1014, 893, 814, 808, 788, 767, 751, 717, 666, 650, 646, 643, 622, 607, 606, 606, 606, 606, 606, 606, 606, 606, 605, 604, 603, 603, 594, 556, 546, 534, 519, 519, 515, 498, 494, 488, 485, 480, 465, 451, 441, 441, 441, 441, 441, 441, 441, 441, 441, 441, 441, 441, 441, 440, 437, 419, 412, 411, 410, 406, 404, 396, 394, 388, 367, 365, 364, 361, 354, 354, 353, 333, 326, 313, 307, 291, 288, 280, 279, 276, 271, 271, 270, 264, 262, 256, 256, 255, 255, 255, 255, 255, 254, 254, 254, 254, 254, 254, 243, 237, 235, 230, 225, 225, 223, 221, 221, 220, 220, 219, 218, 217, 217, 215, 215, 215, 214, 213, 205, 205, 205, 203, 198, 198, 196, 196, 196, 196, 196, 196, 196, 196, 196, 194, 192, 191, 182, 182, 180, 176, 176, 174, 171, 170, 169, 168, 167, 166, 165, 165, 165, 164, 164, 161, 161, 159, 158, 154, 154, 152, 152, 151, 151, 151, 150, 150, 150, 150, 150, 150, 150, 150,

In [18]:
from torch_geometric.loader import NeighborLoader

def train_model(batch):
    model.train()
    optimizer.zero_grad()
    predictions = model(batch.x, batch.edge_index)
    loss = criterion(predictions, batch.y)
    loss.backward()
    optimizer.step()
    return loss.item(), batch.x.size(0)

def evaluate_model(batch):
    model.eval()
    with torch.no_grad():
        predictions = model(batch.x, batch.edge_index)
        pred_labels = predictions.argmax(dim=1)
        correct_predictions = int((pred_labels == batch.y).sum())
    return correct_predictions

losses = []

if gnnTrain:
    loader = NeighborLoader(graph, num_neighbors=[-1, -1], batch_size=5000)
    

    for epoch in range(100):
        total_loss = total_correct = total_nodes = 0

        for batch in loader:
            loss, nodes = train_model(batch)
            total_loss += loss
            total_nodes += nodes
            total_correct += evaluate_model(batch)

        average_loss = total_loss / total_nodes
        accuracy = total_correct / total_nodes

        losses.append(average_loss)
        print(f"Epoch #{epoch}. Training Loss: {average_loss:.5f}, Accuracy: {accuracy:.5f}")
        torch.save(model.state_dict(), gnn_weights)


Epoch #0. Training Loss: 0.00013, Accuracy: 0.82508
Epoch #1. Training Loss: 0.00010, Accuracy: 0.93711
Epoch #2. Training Loss: 0.00009, Accuracy: 0.95051
Epoch #3. Training Loss: 0.00008, Accuracy: 0.95363
Epoch #4. Training Loss: 0.00008, Accuracy: 0.95421
Epoch #5. Training Loss: 0.00008, Accuracy: 0.95589
Epoch #6. Training Loss: 0.00008, Accuracy: 0.95520
Epoch #7. Training Loss: 0.00008, Accuracy: 0.95588
Epoch #8. Training Loss: 0.00008, Accuracy: 0.95671
Epoch #9. Training Loss: 0.00008, Accuracy: 0.95703
Epoch #10. Training Loss: 0.00008, Accuracy: 0.95378
Epoch #11. Training Loss: 0.00008, Accuracy: 0.95518
Epoch #12. Training Loss: 0.00008, Accuracy: 0.95672
Epoch #13. Training Loss: 0.00008, Accuracy: 0.95706
Epoch #14. Training Loss: 0.00008, Accuracy: 0.95686
Epoch #15. Training Loss: 0.00008, Accuracy: 0.95723
Epoch #16. Training Loss: 0.00008, Accuracy: 0.95689
Epoch #17. Training Loss: 0.00008, Accuracy: 0.95572
Epoch #18. Training Loss: 0.00008, Accuracy: 0.95699
Epo

In [19]:
if create_store:
    model.eval()
    out = model.encode(graph.x, graph.edge_index).tolist()
    
    gnn_map = {}
    
    for i in range(len(mapp)):
        gnn_map[lbl[mapp[i]]] = (out[i],list(nemap[mapp[i]]))
    
    # with open("data_files/emb_store.json", "w") as file:
    #     json.dump(gnn_map, file)
        
    with open("emb_store.json", "w") as file:
        json.dump(gnn_map, file)

In [20]:
# with open("data_files/emb_store.json", "r") as file:
#     gnn_map = json.load(file)

In [21]:
with open("emb_store.json", "r") as file:
    gnn_map = json.load(file)

In [22]:
import numpy as np

def load_features(filename=None, similarity=1):
    nodes, y_train, edges, mapp, lbl, nemap = load_data(filename)
    zero_vector = np.zeros(20)

    X_train = []
    for idx, map_item in enumerate(mapp):
        label = lbl[map_item]
        node_feature = nodes[idx]

        if label in gnn_map:
            emb, stored_set = gnn_map[label]
            current_set = nemap[map_item]
            
            if len(current_set) == 0 and len(stored_set) == 0:
                jaccard_similarity = 1.0  
            elif len(current_set) == 0 or len(stored_set) == 0:
                jaccard_similarity = 0.0  
            else:
                intersection = current_set.intersection(stored_set)
                union = current_set.union(stored_set)
                jaccard_similarity = len(intersection) / len(union)

            feature_vector = emb if jaccard_similarity >= similarity else zero_vector
        else:
            feature_vector = zero_vector

        X_train.append(np.hstack((node_feature, feature_vector)))

    return np.array(X_train), y_train, edges, mapp

In [23]:
from sklearn.metrics import accuracy_score
from collections import Counter
import xgboost as xgb

if xgbTrain:
    file_path = '../SysClient0201.systemia.com.txt'
    x,y,_,_ = load_features(file_path)
    

    xgb_cl = xgb.XGBClassifier()

    xgb_cl.fit(x,y)
    pickle.dump(xgb_cl, open(xgboost_weights, "wb"))

    preds = xgb_cl.predict(x)
    print(accuracy_score(y, preds))

[72582, 8448, 6447, 4648, 3292, 2731, 2334, 2326, 2109, 2079, 1385, 1384, 1384, 1384, 1383, 1124, 1124, 1120, 1120, 1120, 1119, 1045, 1021, 1014, 893, 814, 808, 788, 767, 751, 717, 666, 650, 646, 643, 622, 607, 606, 606, 606, 606, 606, 606, 606, 606, 605, 604, 603, 603, 594, 556, 546, 534, 519, 519, 515, 498, 494, 488, 485, 480, 465, 451, 441, 441, 441, 441, 441, 441, 441, 441, 441, 441, 441, 441, 441, 440, 437, 419, 412, 411, 410, 406, 404, 396, 394, 388, 367, 365, 364, 361, 354, 354, 353, 333, 326, 313, 307, 291, 288, 280, 279, 276, 271, 271, 270, 264, 262, 256, 256, 255, 255, 255, 255, 255, 254, 254, 254, 254, 254, 254, 243, 237, 235, 230, 225, 225, 223, 221, 221, 220, 220, 219, 218, 217, 217, 215, 215, 215, 214, 213, 205, 205, 205, 203, 198, 198, 196, 196, 196, 196, 196, 196, 196, 196, 196, 194, 192, 191, 182, 182, 180, 176, 176, 174, 171, 170, 169, 168, 167, 166, 165, 165, 165, 164, 164, 161, 161, 159, 158, 154, 154, 152, 152, 151, 151, 151, 150, 150, 150, 150, 150, 150, 150, 150,

In [24]:
def load_pkl(fname):
    with open(fname, 'rb') as f:
        obj = pickle.load(f)
    return obj

In [25]:
def validate(file_path):
    x,y,_,_ = load_features(file_path)
    xgb_cl = load_pkl(xgboost_weights)

    pred = xgb_cl.predict(x)
    proba = xgb_cl.predict_proba(x)

    sorted = np.sort(proba, axis=1)
    conf = (sorted[:,-1] - sorted[:,-2]) / sorted[:,-1]
    conf = (conf - conf.min()) / conf.max()

    check = (pred == y)
    flag = ~torch.tensor(check)
    scores = conf[flag].tolist()
    return scores

In [26]:
from itertools import compress
from torch_geometric import utils

def Get_Adjacent(ids, mapp, edges, hops):
    if hops == 0:
        return set()
    
    neighbors = set()
    for edge in zip(edges[0], edges[1]):
        if any(mapp[node] in ids for node in edge):
            neighbors.update(mapp[node] for node in edge)

    if hops > 1:
        neighbors = neighbors.union(Get_Adjacent(neighbors, mapp, edges, hops - 1))
    
    return neighbors

def calculate_metrics(TP, FP, FN, TN):
    FPR = FP / (FP + TN) if FP + TN > 0 else 0
    TPR = TP / (TP + FN) if TP + FN > 0 else 0

    prec = TP / (TP + FP) if TP + FP > 0 else 0
    rec = TP / (TP + FN) if TP + FN > 0 else 0
    fscore = (2 * prec * rec) / (prec + rec) if prec + rec > 0 else 0

    return prec, rec, fscore, FPR, TPR

def helper(MP, all_pids, GP, edges, mapp):
    TP = MP.intersection(GP)
    FP = MP - GP
    FN = GP - MP
    TN = all_pids - (GP | MP)

    two_hop_gp = Get_Adjacent(GP, mapp, edges, 2)
    two_hop_tp = Get_Adjacent(TP, mapp, edges, 2)
    FPL = FP - two_hop_gp
    TPL = TP.union(FN.intersection(two_hop_tp))
    FN = FN - two_hop_tp

    TP, FP, FN, TN = len(TPL), len(FPL), len(FN), len(TN)

    prec, rec, fscore, FPR, TPR = calculate_metrics(TP, FP, FN, TN)
    print(f"True Positives: {TP}, True Negatives: {TN}, False Positives: {FP}, False Negatives: {FN}")
    print(f"Precision: {round(prec, 2)}, Recall: {round(rec, 2)}, Fscore: {round(fscore, 2)}")
    
    return TPL, FPL

def calculate_similarity(set1, set2):
    if len(set1) == 0 and len(set2) == 0:
        return 1.0  
    elif len(set1) == 0 or len(set2) == 0:
        return 0.0  
    
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

In [27]:
import numpy as np

def load_features_test(dataframe, similarity_threshold=1):
    nodes, y_train, edges, mapping, label_map, node_entity_map = Featurize(dataframe, csv_path)
    X_train = []

    for i, map_id in enumerate(mapping):
        label = label_map[map_id]
        node_embedding = np.zeros(20)  

        if label in gnn_map:
            embedding, stored_set = gnn_map[label]
            current_set = node_entity_map[map_id]
            similarity_metric = calculate_similarity(current_set, stored_set)

            if similarity_metric >= similarity_threshold:
                node_embedding = np.array(embedding)

        X_train.append(np.hstack((nodes[i], node_embedding)))

    return np.array(X_train), y_train, edges, mapping

In [28]:
import json
import numpy as np
import torch
from torch_geometric import utils

In [29]:
def load_events_from_hosts(hosts):
    all_events = []
    for host in hosts:
        path = f'../SysClient0{host}.systemia.com.txt'
        with open(path, 'r') as file:
            raw_events = [json.loads(line) for line in file]
        all_events.extend(raw_events)
    return all_events

def load_ground_truth(gt_file):
    with open(gt_file, 'r') as file:
        gt_nodes = set(file.read().split())
    return gt_nodes

def evaluate_model(df, xgb_cl, similarity_threshold, confidence_threshold):
    x, y, edges, mapp = load_features_test(df)

    pred = xgb_cl.predict(x)
    proba = xgb_cl.predict_proba(x)

    sorted_proba = np.sort(proba, axis=1)
    conf = (sorted_proba[:, -1] - sorted_proba[:, -2]) / sorted_proba[:, -1]
    normalized_conf = (conf - conf.min()) / conf.max()

    check = (pred == y) & (normalized_conf > confidence_threshold)
    flag = ~torch.tensor(check)

    index = utils.mask_to_index(flag).tolist()
    ids = {mapp[idx] for idx in index}
    return ids,edges,mapp

In [30]:
import json
import numpy as np
import torch

def read_event_data(host):
    file_path = f'../SysClient0{host}.systemia.com.txt'
    with open(file_path, 'r') as file:
        return [json.loads(line) for line in file]
        
def stream_events(batch_size, window_size):
    event_buffer = {}
    hosts = ['051']
    positions = {host: 0 for host in hosts}
    while True:
        for host in hosts:
            if host not in event_buffer or len(event_buffer[host]) < positions[host] + batch_size:
                events = read_event_data(host)
                dframe = transform(events)
                if host in event_buffer:
                    event_buffer[host] = event_buffer[host].append(dframe, ignore_index=True)
                else:
                    event_buffer[host] = dframe
            start = positions[host]
            end = start + batch_size
            yield event_buffer[host][start:end]
            positions[host] += window_size
            if positions[host] >= len(event_buffer[host]):
                return

def analyze_events(data_frame, ground_truth_nodes):
    
    if data_frame['properties'].apply(lambda x: isinstance(x, str)).any():
        data_frame['properties'] = data_frame['properties'].apply(json.loads)
        
    actor_and_object_ids = set(data_frame['actorID']) | set(data_frame['objectID'])
    relevant_ground_truth = {x for x in ground_truth_nodes if x in actor_and_object_ids}

    features, labels, edges, mapping = load_features_test(data_frame)
    model = load_pkl(xgboost_weights)

    predictions = model.predict(features)
    probabilities = model.predict_proba(features)

    sorted_probabilities = np.sort(probabilities, axis=1)
    confidence_scores = (sorted_probabilities[:, -1] - sorted_probabilities[:, -2]) / sorted_probabilities[:, -1]
    normalized_confidence = (confidence_scores - confidence_scores.min()) / confidence_scores.max()

    misclassified = ~torch.tensor(predictions == labels)
    misclassified_indices = utils.mask_to_index(misclassified).tolist()
    misclassified_ids = {mapping[idx] for idx in misclassified_indices}

    helper(misclassified_ids, actor_and_object_ids, relevant_ground_truth, edges, mapping)

In [31]:
def traverse(ids, mapping, edges, hops, visited=None):
    if hops == 0:
        return set()

    if visited is None:
        visited = set()

    neighbors = set()
    for src, dst in zip(edges[0], edges[1]):
        src_mapped, dst_mapped = mapping[src], mapping[dst]

        if (src_mapped in ids and dst_mapped not in visited) or \
           (dst_mapped in ids and src_mapped not in visited):
            neighbors.add(src_mapped)
            neighbors.add(dst_mapped)

        visited.add(src_mapped)
        visited.add(dst_mapped)

    neighbors.difference_update(ids) 
    return ids.union(traverse(neighbors, mapping, edges, hops - 1, visited))

# def load_data(file_path):
#     with open(file_path, 'r') as file:
#         return json.load(file)

def find_connected_alerts(start_alert, mapping, edges, depth, remaining_alerts):
    connected_path = traverse({start_alert}, mapping, edges, depth)
    return connected_path.intersection(remaining_alerts)

def generate_incident_graphs(alerts, edges, mapping, depth):
    incident_graphs = []
    remaining_alerts = set(alerts)

    while remaining_alerts:
        alert = remaining_alerts.pop()
        connected_alerts = find_connected_alerts(alert, mapping, edges, depth, remaining_alerts)

        if len(connected_alerts) > 1:
            incident_graphs.append(connected_alerts)
            remaining_alerts -= connected_alerts

    return incident_graphs

### Testing Flash on OpTC Malicious Upgrade Attack

In [32]:
# all_events = load_events_from_hosts(['051'])

# EnActIds = [x['actorID'] for x in all_events]
# EnObjIds = [x['objectID'] for x in all_events]
# EntitySet = set(EnActIds).union(set(EnObjIds))

# df = transform(all_events)

# gt_nodes = load_ground_truth('optc.txt')
# gt_nodes = [x for x in gt_nodes if x in EntitySet]
# gt_nodes = set(gt_nodes)

# xgboost_model = load_pkl(xgboost_weights)
# identified_ids,edges,mapp = evaluate_model(df, xgboost_model, 1, 0.6)

# alerts = helper(identified_ids, EntitySet, gt_nodes, edges, mapp)

### Testing Flash on OpTC Plain PowerShell Empire Attack

In [33]:
all_events = load_events_from_hosts(['201'])

EnActIds = [x['actorID'] for x in all_events]
EnObjIds = [x['objectID'] for x in all_events]
EntitySet = set(EnActIds).union(set(EnObjIds))

df = transform(all_events)

gt_nodes = load_ground_truth('../optc.txt')
gt_nodes = [x for x in gt_nodes if x in EntitySet]
gt_nodes = set(gt_nodes)

xgboost_model = load_pkl(xgboost_weights)
identified_ids,edges,mapp = evaluate_model(df, xgboost_model, 1, 0)

alerts = helper(identified_ids, EntitySet, gt_nodes, edges, mapp)

[72582, 8448, 6447, 4648, 3292, 2731, 2334, 2326, 2109, 2079, 1385, 1384, 1384, 1384, 1383, 1124, 1124, 1120, 1120, 1120, 1119, 1045, 1021, 1014, 893, 814, 808, 788, 767, 751, 717, 666, 650, 646, 643, 622, 607, 606, 606, 606, 606, 606, 606, 606, 606, 605, 604, 603, 603, 594, 556, 546, 534, 519, 519, 515, 498, 494, 488, 485, 480, 465, 451, 441, 441, 441, 441, 441, 441, 441, 441, 441, 441, 441, 441, 441, 440, 437, 419, 412, 411, 410, 406, 404, 396, 394, 388, 367, 365, 364, 361, 354, 354, 353, 333, 326, 313, 307, 291, 288, 280, 279, 276, 271, 271, 270, 264, 262, 256, 256, 255, 255, 255, 255, 255, 254, 254, 254, 254, 254, 254, 243, 237, 235, 230, 225, 225, 223, 221, 221, 220, 220, 219, 218, 217, 217, 215, 215, 215, 214, 213, 205, 205, 205, 203, 198, 198, 196, 196, 196, 196, 196, 196, 196, 196, 196, 194, 192, 191, 182, 182, 180, 176, 176, 174, 171, 170, 169, 168, 167, 166, 165, 165, 165, 164, 164, 161, 161, 159, 158, 154, 154, 152, 152, 151, 151, 151, 150, 150, 150, 150, 150, 150, 150, 150,

### Testing Flash on OpTC Custom PowerShell Empire Attack

In [34]:
# all_events = load_events_from_hosts(['501'])

# EnActIds = [x['actorID'] for x in all_events]
# EnObjIds = [x['objectID'] for x in all_events]
# EntitySet = set(EnActIds).union(set(EnObjIds))

# df = transform(all_events)

# gt_nodes = load_ground_truth('optc.txt')
# gt_nodes = [x for x in gt_nodes if x in EntitySet]
# gt_nodes = set(gt_nodes)

# xgboost_model = load_pkl(xgboost_weights)
# identified_ids,edges,mapp = evaluate_model(df, xgboost_model, 1, 0.98)

# alerts = helper(identified_ids, EntitySet, gt_nodes, edges, mapp)

### Testing Flash on Streaming Batches Generated from OpTC Attack Logs.

In [35]:
stream = False
if stream:
    for data_frame in stream_events(250000, 250):
        gt_nodes = load_ground_truth('optc.txt')
        analyze_events(data_frame, gt_nodes)